### Libraries

In [1]:
%%capture
%reset -f                        # clear all variables from the workspace
'generic imports'
import os                              
import pandas as pd                     
import numpy as np
import datetime
import re
import sys
sys.path.append(os.path.abspath('..'))
from src import utils
import importlib
importlib.reload(utils)  

'machine learning library imports'
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

### Load data

In [2]:
AUGMENTATION = 'GReaT' # options: 'None', 'GReat', 'SMOTE', 'SMOTE-NC' or 'RealTabFormer'

data_dir = os.path.abspath('../data')

# Load the train and test datasets
df_train, df_test = utils.load_dataset(data_directory=data_dir, 
                                       augmentation=AUGMENTATION, 
                                       ignore_columns=['mbtcp.unit_id', 
                                                       'mbtcp.trans_id']) 

Loading complete.
Train data: 1500000 rows, 46 columns. 
Test data: 381934 rows, 46 columns.


### Data preparation

In [3]:
# Creates X_train, y_train
X_train = df_train.drop(['Attack_label', 'Attack_type'], axis=1)
y_train = df_train['Attack_type']

# Creates X_test, y_test
X_test = df_test.drop(['Attack_label', 'Attack_type'], axis=1)
y_test = df_test['Attack_type']

#### Convert categorical features to one-hot encoded features

In [4]:
# Encode the training and test labels if needed
X_train_enc, X_test_enc, info = utils.encode_categorical(X_train, X_test)

Categorical features to be encoded:

dns.qry.name.len
mqtt.topic
http.referer
mqtt.conack.flags
mqtt.protoname
http.request.method
http.request.version

Encoding complete.
No of features before encoding: 44
No of features after encoding: 100


In [ ]:
valid = False

if valid == True:
    def get_unique_values(df, columns):
        unique_values = df[columns].apply(lambda x: ''.join(x.astype(str)), axis=1).unique()
        unique_values.sort()
        return unique_values

    http_referer_columns = [col for col in X_train.columns if 'http.referer' in col]
    http_request_version_columns = [col for col in X_train.columns if 'http.request.version' in col]
    dns_qry_name_len_columns = [col for col in X_train.columns if 'dns.qry.name.len' in col]
    mqtt_conack_flags_columns = [col for col in X_train.columns if 'mqtt.conack.flags' in col]
    mqtt_protoname_columns = [col for col in X_train.columns if 'mqtt.protoname' in col]
    http_request_method_columns = [col for col in X_train.columns if 'http.request.method' in col]

    unique_values_http_referer = get_unique_values(X_train, http_referer_columns)
    unique_values_http_request_version = get_unique_values(X_train, http_request_version_columns)
    unique_values_dns_qry_name_len = get_unique_values(X_train, dns_qry_name_len_columns)
    unique_values_mqtt_conack_flags = get_unique_values(X_train, mqtt_conack_flags_columns)
    unique_values_mqtt_protoname = get_unique_values(X_train, mqtt_protoname_columns)

    # Apply the same function to X_test
    unique_values_http_referer_test = get_unique_values(X_test, http_referer_columns)
    unique_values_http_request_version_test = get_unique_values(X_test, http_request_version_columns)
    unique_values_dns_qry_name_len_test = get_unique_values(X_test, dns_qry_name_len_columns)
    unique_values_mqtt_conack_flags_test = get_unique_values(X_test, mqtt_conack_flags_columns)
    unique_values_mqtt_protoname_test = get_unique_values(X_test, mqtt_protoname_columns)
    
    # Join the unique values from both train and test data and print them
    unique_values_http_referer = np.unique(np.concatenate((unique_values_http_referer, unique_values_http_referer_test)))
    unique_values_http_request_version = np.unique(np.concatenate((unique_values_http_request_version, unique_values_http_request_version_test)))
    unique_values_dns_qry_name_len = np.unique(np.concatenate((unique_values_dns_qry_name_len, unique_values_dns_qry_name_len_test)))
    unique_values_mqtt_conack_flags = np.unique(np.concatenate((unique_values_mqtt_conack_flags, unique_values_mqtt_conack_flags_test)))
    unique_values_mqtt_protoname = np.unique(np.concatenate((unique_values_mqtt_protoname, unique_values_mqtt_protoname_test)))

    print(f'Unique values for http.referer: \n{unique_values_http_referer}\n')
    print(f'Unique values for http.request.version: \n{unique_values_http_request_version}\n')
    print(f'Unique values for dns.qry.name.len: \n{unique_values_dns_qry_name_len}\n')
    print(f'Unique values for mqtt.conack.flags: \n{unique_values_mqtt_conack_flags}\n')
    print(f'Unique values for mqtt.protoname: \n{unique_values_mqtt_protoname}\n')
    
    # check if X_train and X_test have categorical features
    print(f'X_train categorical features: {X_train.select_dtypes(include="object").columns}')
    print(f'X_test categorical features: {X_test.select_dtypes(include="object").columns}')

#### Columns name validation for XGBoost

In [5]:
# define a regular expression pattern to match (),:; {}><[]$- 
pattern = r'[(),:;{}><\[\]\$\-]|_{2,}|\s+'

# clean X_train  column names using the regular expression pattern
cleaned_columns = [re.sub(pattern, '', col) for col in X_train_enc.columns]

# check for duplicates and if they exist, add suffix to make them unique
unique_names = []
for name in cleaned_columns:
    if name in unique_names:
        # add suffix to make the name unique
        suffix = 1
        while f"{name}_{suffix}" in unique_names:
            suffix += 1
        unique_names.append(f"{name}_{suffix}")
    else:
        unique_names.append(name)

# Replace the column names with the cleaned and unique names
X_train_enc.columns, X_test_enc.columns = unique_names, unique_names


# check if any of X_train column names have [, ] or <, > characters
for name in X_train_enc.columns:
    if '[' in name or ']' in name or '<' in name or '>' in name:
        print(name)

# Assert all X_train_enc and X_test_enc column names are unique
assert len(np.unique(X_train_enc.columns)) == len(X_train_enc.columns)
assert len(np.unique(X_test_enc.columns)) == len(X_test_enc.columns)

#### Label encoding

In [6]:
y_train_enc, y_test_enc, le = utils.encode_labels(y_train, y_test)

Attack_type and encoded labels:

Backdoor                0
DDoS_HTTP               1
DDoS_ICMP               2
DDoS_TCP                3
DDoS_UDP                4
Fingerprinting          5
MITM                    6
Normal                  7
Password                8
Port_Scanning           9
Ransomware              10
SQL_injection           11
Uploading               12
Vulnerability_scanner   13
XSS                     14


### Model training

In [7]:
# Shuffle training data
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Instantiate the XGBClassifier
xgb = XGBClassifier(random_state=42,
                    seed=42,
                    num_class= (le.classes_).size)

# Train the model
xgb_clf =  xgb.fit(X_train_enc, y_train_enc)

### Model Evaluation

In [8]:
predictions = xgb_clf.predict(X_test_enc)

In [9]:
accuracy = metrics.accuracy_score(y_test_enc, predictions)
precision_w = metrics.precision_score(y_test_enc, predictions, average='weighted', zero_division=1)
recall_w = metrics.recall_score(y_test_enc, predictions, average='weighted')
f1_score_w = metrics.f1_score(y_test_enc, predictions, average='weighted')
precision_m = metrics.precision_score(y_test_enc, predictions, average='macro', zero_division=1)
recall_m = metrics.recall_score(y_test_enc, predictions, average='macro')
f1_score_m = metrics.f1_score(y_test_enc, predictions, average='macro')

In [10]:
# Create dictionary for results
results = {
    "model": "XGBoost",
    "augmentations": AUGMENTATION,
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "accuracy": accuracy,
    "precision_macro": precision_m,
    "recall_macro": recall_m,
    "f1_macro": f1_score_m,
    "precision_weighted": precision_w,
    "recall_weighted": recall_w,
    "f1_weighted": f1_score_w
    }

utils.print_results_table(results)

╒══════════════════════╤═════════╕
│ Metric               │ Value   │
╞══════════════════════╪═════════╡
│ Accuracy             │ 98.14%  │
├──────────────────────┼─────────┤
│ Precision (macro)    │ 82.43%  │
├──────────────────────┼─────────┤
│ Recall (macro)       │ 91.19%  │
├──────────────────────┼─────────┤
│ F1 (macro)           │ 82.90%  │
├──────────────────────┼─────────┤
│ Precision (weighted) │ 98.52%  │
├──────────────────────┼─────────┤
│ Recall (weighted)    │ 98.14%  │
├──────────────────────┼─────────┤
│ F1 (weighted)        │ 98.29%  │
╘══════════════════════╧═════════╛


#### Save Metrics Results 

In [11]:
# Save results to csv   
utils.save_results_to_csv([results], '../results/metrics/xgboost.csv')

#### Confusion Matrix

In [12]:
conf_mat = metrics.confusion_matrix(y_test_enc, predictions)

attack_labels = ['Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP', 
'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning', 'Ransomware', 
'SQL_injection', 'Uploading', 'Vulnerability_scanner', 'XSS']

# Create a dataframe from the confusion matrix
conf_mat_df = pd.DataFrame(conf_mat, 
                            index = attack_labels, 
                            columns = attack_labels)
conf_mat_df.index.name = 'Actual'
conf_mat_df.columns.name = 'Predicted'

# Save the confusion matrix
conf_mat_df.to_csv(f"../results/conf_matrix/{results['model']}_{results['augmentations']}.csv")
conf_mat_df

Predicted,Backdoor,DDoS_HTTP,DDoS_ICMP,DDoS_TCP,DDoS_UDP,Fingerprinting,MITM,Normal,Password,Port_Scanning,Ransomware,SQL_injection,Uploading,Vulnerability_scanner,XSS
Actual,,,,,,,,,,,,,,,
Backdoor,4469,0,0,0,0,1,147,0,0,154,9,0,0,0,2
DDoS_HTTP,0,8790,0,0,0,0,0,0,0,0,0,0,0,70,768
DDoS_ICMP,0,0,13459,0,0,42,0,0,0,0,0,0,0,0,0
DDoS_TCP,0,0,0,10009,0,0,0,0,0,0,0,0,0,0,0
DDoS_UDP,0,0,0,0,24590,11,0,0,0,0,0,0,0,0,0
Fingerprinting,0,0,11,0,0,87,23,0,0,24,1,0,0,0,0
MITM,0,0,0,0,0,0,76,0,0,0,0,0,0,0,0
Normal,0,0,0,0,0,0,0,272776,0,0,0,0,0,0,0
Password,0,1,0,0,0,269,188,0,8216,0,0,696,738,0,0
